In [1]:
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import pandas as pd
import time
from tqdm import tnrange
from urllib.request import urlopen
import re
import requests
import urllib.request
from tqdm import tqdm
from bs4 import BeautifulSoup
from sklearn import preprocessing
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
df = pd.read_csv('data/train.csv')
le = preprocessing.LabelEncoder()
le.fit(df['cat3'].values)
df['cat3'] = le.transform(df['cat3'].values)

le = preprocessing.LabelEncoder()
le.fit(df['cat2'].values)
df['cat2'] = le.transform(df['cat2'].values)
le = preprocessing.LabelEncoder()
le.fit(df['cat1'].values)
df['cat1'] = le.transform(df['cat1'].values)

In [3]:
folds = StratifiedKFold(n_splits=5, random_state=20, shuffle=True)
df['kfold'] = -1
for i in range(5):
    df_idx, valid_idx = list(folds.split(df.values, df['cat3']))[i]
    valid = df.iloc[valid_idx]

    df.loc[df[df.id.isin(valid.id) == True].index.to_list(), 'kfold'] = i

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in 

In [4]:
def preprocess_sentence(sentence):
    sent = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    if sent == '':
        sent = re.sub(r'\<[^>]*\>', '', sentence) # <> 괄호로 닫힌 문자열  제거
    sentence = re.sub(r'\([^)]*\)', '', sent) # () 괄호로 닫힌 문자열  제거
    sentence = re.sub('\n+', ' ', sentence)
    sentence = re.sub('\s+', ' ', sentence)
    sentence = re.sub('※', '', sentence)
    sentence = re.sub('#', '', sentence)
    sentence = re.sub('\*', '', sentence)
    sentence = re.sub('\'', '', sentence)
    sentence = sentence.lstrip()
    
    return sentence

In [5]:
def chrome_setting():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    return driver

In [6]:
driver=chrome_setting()

In [7]:
def kor_to_trans(driver, text_data, trans_lang):
    """
    trans_lang에 넣는 파라미터 값:
    'en' -> 영어
    """
    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
    trans_list = []
    for i in tqdm(range(len(text_data))):
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            trans_list.append(backtrans)
        except:
            try:
                driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang)
                driver.find_element_by_xpath('//*[@id="txtSource"]').send_keys(text_data[i])
                time.sleep(2.0)
                element = WebDriverWait(driver, 10).until(target_present)
                backtrans = element.text
                
                if (backtrans == '') | (backtrans == ' '):
                    element=WebDriverWait(driver, 10).until(target_present)
                    backtrans = element.text 
                    trans_list.append(backtrans)
                else:
                    trans_list.append(backtrans)
            except:
                trans_list.append('')
    return trans_list

def trans_to_kor(driver, transed_list, transed_lang):
    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
    backtrans_list = []
    for i in tqdm(range(len(transed_list))):
        try:
            driver.get('https://papago.naver.com/?sk='+transed_lang+'&tk=ko&st='+transed_list[i])
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            backtrans_list.append(backtrans)
        except:
            try:
                driver.get('https://papago.naver.com/?sk='+transed_lang+'&tk=ko')
                driver.find_element_by_xpath('//*[@id="txtSource"]').send_keys(transed_list[i])
                time.sleep(2.0)
                element = WebDriverWait(driver, 10).until(target_present)
                backtrans = element.text
                
                if (backtrans == '') | (backtrans == ' '):
                    element=WebDriverWait(driver, 10).until(target_present)
                    backtrans = element.text 
                    backtrans_list.append(backtrans)
                else:
                    backtrans_list.append(backtrans)
            except:
                backtrans_list.append('')
    return backtrans_list


In [8]:
u, count = np.unique(df['cat3'].values, return_counts=True)
df['back_en'] = ''
df['back_ja'] = ''

In [9]:
back_arr = []
empty_idx = []
cnt = 1
for i in range(len(df['overview'])):
    if count[df['cat3'][i]] < 500:
        continue
    if count[df['cat3'][i]] < 650:
        print(cnt)
        txt = df['overview'][i]
        txt = preprocess_sentence(txt)
        if txt == '':
            empty_idx.append(i)
            txt = df['overview'][i]
        mid1 = kor_to_trans(driver, [txt], 'en')
        if mid1 == '':
            empty_idx.append(i)
            continue
        res1 = trans_to_kor(driver, mid1, 'en')
        if res1 == '':
            empty_idx.append(i)
            continue
        mid2 = kor_to_trans(driver, [txt], 'ja')
        if mid2 == '':
            empty_idx.append(i)
            continue
        res2 = trans_to_kor(driver, mid2, 'ja')
        if res2 == '':
            empty_idx.append(i)
            continue
            
        df['overview'][i] = txt
        df['back_en'][i] = res1[0]
        df['back_ja'][i] = res2[0]
        back_arr.append(res1[0])
        cnt += 1

  0%|          | 0/1 [00:00<?, ?it/s]

1


100%|██████████| 1/1 [00:02<00:00,  2.63s/it]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  0%|          | 0/1 [00:00<?, ?it/s]

2


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

1170


  0%|          | 0/1 [00:00<?, ?it/s]

1171


100%|██████████| 1/1 [00:02<00:00,  2.66s/it]


In [12]:
len(empty_idx)

0

In [2]:
df = pd.read_csv('data/train_folds_125.csv')
df2 = pd.read_csv('data/train_folds_125to250.csv')

a1= df['back_en'].to_numpy()
a2= df['back_ja'].to_numpy()
b1= df2['back_en'].to_numpy()
b2= df2['back_ja'].to_numpy()

In [11]:
df = pd.read_csv('data/train_folds_250.csv')
a1= df['back_en'].to_numpy()
a2= df['back_ja'].to_numpy()

In [8]:
u, count = np.unique(df['cat3'].values, return_counts=True)
count_sort_ind = np.argsort(-count)
print(u[count_sort_ind])
print(count[count_sort_ind])

[118  73  41  85  90  52  86 115 119  31  20  44  11  54 112  97  53  57
  84 100  93 121  37   8  96 111   0   5  91  16 120  12   9  38  94  34
  58  82   3  61  60  25   7 127  59   4  18  40  19  99  15 114  67 126
  42  77  17  55  21  10 116  39 113   6 101  92 107  72 125  81  43  80
  63  71  35 122  48  28  32 105  13  69  33  26  83  27  29  74  23 123
  24  95  66  56  79  87  14 106  62 102  70 117  51 124  49 110  78 104
  64  98  88  30  68 109  75   1  46  45  50  76 103  47   2  22  89 108
  65  36]
[3438 1558  785  622  549  495  467  445  404  340  332  310  271  270
  246  243  239  226  211  209  207  207  204  179  173  172  165  160
  151  148  134  126  120  119  118  113  111  105  104   95   95   92
   92   88   85   83   79   76   71   65   64   61   60   57   57   53
   53   43   41   41   40   39   37   36   36   35   34   34   30   30
   29   27   27   27   26   25   24   24   23   22   21   21   20   20
   20   18   18   17   17   15   15   13   12   12   